In [44]:
from ipyparallel import Client
import os
import pprint
pp = pprint.PrettyPrinter(indent=4)
rc = Client()
data_path = os.path.abspath("data")
res_path = os.path.abspath("results")
random_path = os.path.abspath("random")
dview = rc.load_balanced_view()
#dview = rc.direct_view('all')
print 'n of workers:', len(dview)
print len(rc)
pp.pprint(dview.queue_status())

n of workers: 4
4
{   0: {   'completed': 15, 'queue': 0, 'tasks': 0},
    1: {   'completed': 15, 'queue': 0, 'tasks': 0},
    2: {   'completed': 15, 'queue': 0, 'tasks': 0},
    3: {   'completed': 15, 'queue': 0, 'tasks': 0},
    'unassigned': 0}


In [81]:
%%px --local
import os
import pandas as pd
from string import strip
import numpy as np
from string import strip
from itertools import combinations
from scipy import stats
import time

def process(batch):
    print 'start'
    res = [ [str(stats.pearsonr(values_df.loc[n[0]], values_df.loc[n[1]])[0]),str(n[2]),str(n[3])] for n in batch]
    return res

In [82]:
values_df = pd.DataFrame.from_csv(os.path.join(data_path,'in_df.txt'),sep='\t')
cluster_res = pd.DataFrame.from_csv('res_clustering.csv')
print values_df.iloc[:3,:3]
print cluster_res.iloc[:3,:3]
pairs = []
for index, cut_distance in enumerate(cluster_res.columns):
    temp_df = pd.concat([values_df,cluster_res[cut_distance]],1)
    n_of_clusters = cluster_res[cut_distance].value_counts()
    n_of_clusters = n_of_clusters[n_of_clusters>=2]
    for cluster_id in n_of_clusters.index.values:
        genes = temp_df[temp_df[cut_distance]==cluster_id].index.values
        temp_pairs = list(combinations(genes,2))
        temp_pairs = [(n[0],n[1],cut_distance,cluster_id) for n in temp_pairs]
        pairs+=temp_pairs
print len(pairs)


                  median_1     median_2     median_3
Tb927.10.3210  688310000.0  276010000.0  215760000.0
Tb927.5.2310           0.0          0.0          0.0
Tb927.8.7570           0.0          0.0          0.0
               clusters_0.0  clusters_0.06  clusters_0.09
Tb927.10.3210          2325           2324           2323
Tb927.5.2310           1801           1800           1800
Tb927.8.7570           3018           3017           3016
29319788


In [83]:
split = lambda lst, sz: [lst[i:i+sz] for i in range(0, len(lst), sz)]
batches = [n for n in split(pairs, 10000)]
print len(batches)

293198


In [84]:
rc[:].push(dict(
    values_df=values_df       
))

start_time = time.time()
tasks = []
for batch in batches[0:4]:
    ar = dview.apply_async(process, batch)
    tasks.append(ar)
print '-------send to-------------\n'     
print time.time() - start_time
print 'min: ', (time.time() - start_time)/60
print 'h: ', (time.time() - start_time)/60/60
print '-------------------------\n'
  

-------send to-------------

0.0169999599457
min:  0.000299998124441
h:  4.99996874068e-06
-------------------------



In [85]:
pp.pprint(dview.queue_status())
print len(tasks)
print tasks[0].get()[0]

{   0: {   'completed': 112, 'queue': 0, 'tasks': 1},
    1: {   'completed': 113, 'queue': 1, 'tasks': 1},
    2: {   'completed': 113, 'queue': 1, 'tasks': 1},
    3: {   'completed': 113, 'queue': 0, 'tasks': 1},
    'unassigned': 0}
4
['0.999387723211', 'clusters_0.06', '54']


In [86]:
tasks = [n.get() for  n in tasks]
file_res  = open('pearson_res.txt','w')
for batch in tasks:
    for item in batch:
       file_res.write('\t'.join(item)+'\n') 
file_res.close()        

In [90]:
df = pd.read_table('perason_res.txt',sep='\t',header =None)
df.columns = ['person','cut_distance','cluster_id']
df['cut_distance']=[float(n.split('_')[1]) for n in df['cut_distance']]
df.sort_values(by='cut_distance', ascending=True, inplace=True)
print df.head()
combined = df.groupby('cut_distance').mean()
del combined['cluster_id']

          0              1     2
0  0.999388  clusters_0.06    54
1  0.998297  clusters_0.09  2270
2  0.999388  clusters_0.09    54
3  0.999388   clusters_0.1    54
4  0.997761   clusters_0.1  1719


In [97]:
del combined['cluster_id']
combined.to_csv('reduced_pearson.csv')